In [1]:
import numpy as np
import itertools
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import minimize
import pmlb
from sklearn.model_selection import train_test_split
import sympy as sp
import pandas as pd
from collections import OrderedDict
from tqdm import tqdm
import time
from simpleq import SimplEq

In [2]:
all_feynman_datasets = [name for name in pmlb.dataset_names if ('feynman' in name)]

In [3]:
eq_df = pd.read_csv('equations/FeynmanEquations.csv')
bonus_eq_df = pd.read_csv('equations/BonusEquations.csv')
eq_df.dropna(axis = 0, how = 'all', inplace = True)
bonus_eq_df.dropna(axis = 0, how = 'all', inplace = True)
eq_df.Filename = eq_df.Filename.apply(lambda x: 'feynman_' + x.replace('.', '_'))
bonus_eq_df.Filename = bonus_eq_df.Filename.apply(lambda x: 'feynman_' + x.replace('.', '_'))

eq_df = pd.concat([eq_df[['Filename', 'Formula']], bonus_eq_df[['Filename', 'Formula']]])

#feynman_I_15_10 in pmlb equal to I.15.1 in original source
eq_df.Filename = eq_df.Filename.apply(lambda x: x.replace('feynman_I_15_1', 'feynman_I_15_10'))
#replace arc functions for compatibility with sympy
eq_df.Formula = eq_df.Formula.apply(lambda x: x.replace('arcsin', 'asin'))
eq_df.Formula = eq_df.Formula.apply(lambda x: x.replace('arccos', 'acos'))
eq_df.Formula = eq_df.Formula.apply(lambda x: x.replace('arctan', 'atan'))

In [4]:
datasets={}
for name in tqdm(all_feynman_datasets):
    try:
        datasets[name] = pmlb.fetch_data(name, local_cache_dir='pmlb_data')
    except Exception as e:
        print("Did not work for", name)

 85%|████████▍ | 101/119 [00:15<00:10,  1.74it/s]

Did not work for feynman_test_1


100%|██████████| 119/119 [00:20<00:00,  5.94it/s]


In [5]:
simpleq = SimplEq()

In [6]:
# Suppreses warnings
import warnings
warnings.filterwarnings("ignore")

gen = np.random.default_rng(0)

results = pd.DataFrame(columns=['dataset','equal','score', 'time', 'original', 'equation'])
for i, name in enumerate(datasets.keys()):
# for name in datasets:
    print(i, name)
    df = datasets[name]
    X = df.drop('target', axis=1).to_numpy()
    y = df[['target']].to_numpy().ravel()

    # Subsample
    n_samples = 10000
    idx = gen.choice(X.shape[0], n_samples, replace=False)
    X = X[idx]
    y = y[idx]
    
    # Randomly divide into training and test data using sklearn
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    time_start = time.time()
    simpleq.fit(X_train, y_train)
    f = simpleq.function
    expr = simpleq.expr
    time_end = time.time()

    # Calculate R2 score
    y_pred = f(X_test)
    r2 = r2_score(y_test, y_pred)

    print(f"Expr: {expr}, R2: {r2}, Time: {time_end-time_start}")

    orig = eq_df[eq_df.Filename == name]['Formula'].tolist()
    if (len(orig) == 0):
        print(f'equation {name} not found')

    results.loc[len(results)] = [name, None, r2, time_end-time_start, orig[0], str(expr)]


0 feynman_III_10_19
Expr: 2.3493855032298*1.10485484286908**x3*x1**0.998215701342678*x2**0.259843968812752*x4**0.260076184385527, R2: 0.9757810864266306, Time: 14.437185287475586
1 feynman_III_12_43
Expr: 0.479033858250809*log(x1) + 0.483820749273349*log(x2) - 1.45655654593359, R2: 0.932335788996308, Time: 1.0897924900054932
2 feynman_III_13_18
Expr: 13.7238621614379*1.37703114247861**x3*x1**0.998498610220785*x2**1.99207423390297/x4**0.999747796199983, R2: 0.9940944931409267, Time: 20.151264429092407
3 feynman_III_14_14
Expr: 0.134312915144884*3.93782040276466**x3*4.00278492401421**x2*x1**0.998672288961675/(x4**2.16984871431557*x5**2.1719374511407), R2: 0.9816955350031991, Time: 46.71353101730347
4 feynman_III_15_12
Expr: 3.11724316813843*x1**1.00450618741419/(x2**0.179896877207684*x3**0.233341202901151), R2: 0.24800965832581978, Time: 4.339467525482178
5 feynman_III_15_14
Expr: -0.0105301524080613*x2 - 0.0222557243893898*x3 + 0.00588240000084792*log(x1) + 0.025827641564285, R2: 0.5867

In [7]:
results

,dataset,equal,score,time,original,equation
0,feynman_III_10_19,None,0.975781,14.437185,mom*sqrt(Bx**2+By**2+Bz**2),2.3493855032298*1.10485484286908**x3*x1**0.998...
1,feynman_III_12_43,None,0.932336,1.089792,n*(h/(2*pi)),0.479033858250809*log(x1) + 0.483820749273349*...
2,feynman_III_13_18,None,0.994094,20.151264,2*E_n*d**2*k/(h/(2*pi)),13.7238621614379*1.37703114247861**x3*x1**0.99...
3,feynman_III_14_14,None,0.981696,46.713531,I_0*(exp(q*Volt/(kb*T))-1),0.134312915144884*3.93782040276466**x3*4.00278...
4,feynman_III_15_12,None,0.248010,4.339468,2*U*(1-cos(k*d)),3.11724316813843*x1**1.00450618741419/(x2**0.1...
...,...,...,...,...,...,...
113,feynman_test_5,None,0.998845,14.371841,2*pi*d**(3/2)/sqrt(G*(m1+m2)),4.86651286802567*0.875069488610412**x3*x1**1.5...
114,feynman_test_6,None,0.672723,265.273836,sqrt(1+2*epsilon**2*E_n*L**2/(m*(Z_1*Z_2*q**2)...,1.69213541777815*1.19785902922133**x2*x1**0.35...
115,feynman_test_7,None,0.997861,45.184654,sqrt(8*pi*G*rho/3-alpha*c**2/d**2),2.84222902831902*0.975996064427683**x4*0.98749...
116,feynman_test_8,None,0.923161,13.501449,E_n/(1+E_n/(m*c**2)*(1-cos(theta))),0.37492826948651*x2 + 0.782408643530506*x3 - 0...


In [8]:
# Create a folder for results if it does not exist
import os
if not os.path.exists('results'):
    os.makedirs('results')

In [9]:
# Save to csv
results.to_csv('results/results.csv', index=False)

In [11]:
results = pd.read_csv('results/results.csv')

The code for testing the equality between the two expressions is inspired by the one found in this Kaggle notebook: https://www.kaggle.com/code/jano123/feynman-equations-by-symbolic-regression

In [12]:
def simplify(expr, **args):
    try:
        expr2 = sp.simplify(expr, **args)
    except Exception as e:
        print('simplify ' + str(e))
        return expr
    return expr2

def round_floats(ex1, precision=4):
    ex2 = ex1
    for a in sp.preorder_traversal(ex1):
        if isinstance(a, sp.Float):
            ex2 = ex2.subs(a, sp.Float(round(a, precision),precision))
    return ex2

def get_eq(X: pd.DataFrame, expr: str):
    features = [c for c in X.columns]
    
    eq = round_floats(sp.parse_expr(expr))
    model_str = str(simplify(eq, ratio=1))
    #mapping1 = {'x'+str(i+1): '$$$'+str(i+1) for i, k in enumerate(X.columns)}
    #mapping2 = {'$$$'+str(i+1): k for i, k in enumerate(X.columns)}
    mapping1 = OrderedDict({'x'+str(i+1): '$$$'+str(i+1) for i, k in enumerate(X.columns)})
    mapping2 = OrderedDict({'$$$'+str(i+1): k for i, k in enumerate(X.columns)})
    new_model = model_str
    for k, v in reversed(mapping1.items()):
        new_model = new_model.replace(k, v)
    for k, v in reversed(mapping2.items()):
        new_model = new_model.replace(k, v)
    return round_floats(sp.parse_expr(new_model, local_dict = {k:sp.Symbol(k) for k in features}))

def get_sym_model(X, expr, return_str=True):
    features = [c for c in X.columns if c != 'target']
    model_str = expr.replace('pi','3.1415926535')
    if return_str:
        return model_str

    model_sym = sp.parse_expr(model_str,local_dict = {k:sp.Symbol(k) for k in features})
    model_sym = round_floats(model_sym)
    return model_sym

def compare_sym_model(X: pd.DataFrame, expr1, expr2):
    model1 = expr1
    model2 = expr2

    try:
        sym_diff = round_floats(model1 - model2)
        sym_frac = round_floats(model1/model2)
        #print('sym_diff:',sym_diff)
        #print('sym_frac:',sym_frac)
        # check if we can skip simplification 
        if not sym_diff.is_constant() or sym_frac.is_constant():
            sym_diff = round_floats(simplify(sym_diff, ratio=1))
            #print('simplified sym_diff:',sym_diff)

        symbolic_error_is_zero = str(sym_diff) == '0'
        symbolic_error_is_constant = sym_diff.is_constant()
        symbolic_fraction_is_constant = sym_frac.is_constant()
    except Exception as e:
        print(e)
        return False
        
    return symbolic_error_is_zero or symbolic_error_is_constant or symbolic_fraction_is_constant


In [13]:

for index, row in results.iterrows():
    X = pmlb.fetch_data(row["dataset"], local_cache_dir="pmlb_data").drop('target', axis=1)
    # skip inaccurate results
    if row['score'] < 0.99:
        results.at[index,'equation'] = str(get_eq(X, row['equation']))
        results.at[index,'equal'] = False
        continue
    
    eq = get_eq(X, row['equation'])
    orig_eq = get_sym_model(X, row['original'], False)
    
    equal = compare_sym_model(X, eq, orig_eq)
    results.at[index,'equation'] = eq
    results.at[index,'equal'] = equal
    
    print(f'{index} {row["dataset"]} equal: {equal} [{eq}]')

results.head()

2 feynman_III_13_18 equal: False [13.72*1.377**k*E_n**0.9985*d**1.992/h**0.9997]
6 feynman_III_15_27 equal: True [6.283*alpha**1.0/(d**1.0*n**1.0)]
10 feynman_III_4_32 equal: False [6.261*1.393**kb*T**1.036/(h**1.032*omega**1.027)]
11 feynman_III_4_33 equal: False [0.8777*0.9807**h*0.9808**omega*T**1.072*kb**1.072]
12 feynman_III_7_38 equal: True [12.57*B**1.0*mom**1.0/h**1.0]
16 feynman_II_11_20 equal: False [0.3621*1.375**n_rho*Ef**0.9939*p_d**1.999/(T**0.9884*kb**1.001)]
17 feynman_II_11_27 equal: False [1.399*Ef**1.004*alpha**1.187*epsilon**0.9972*n**1.188]
19 feynman_II_11_3 equal: False [1.407*1.383**omega*Ef**1.005*q**1.004/(m**1.001*omega_0**2.486)]
21 feynman_II_13_23 equal: False [-0.2988*c + 1.044*log(rho_c_0) + 0.1826*log(v) + 0.2699]
26 feynman_II_24_17 equal: False [1.423*1.248**omega*d**0.1115/c**1.111]
27 feynman_II_27_16 equal: True [1.0*Ef**2.0*c**1.0*epsilon**1.0]
28 feynman_II_27_18 equal: True [1.0*Ef**2.0*epsilon**1.0]
33 feynman_II_34_29b equal: False [6.788*1.37

,dataset,equal,score,time,original,equation
0,feynman_III_10_19,False,0.975781,14.437185,mom*sqrt(Bx**2+By**2+Bz**2),2.349*1.105**By*Bx**0.2598*Bz**0.2601*mom**0.9982
1,feynman_III_12_43,False,0.932336,1.089792,n*(h/(2*pi)),0.4838*log(h) + 0.479*log(n) - 1.457
2,feynman_III_13_18,False,0.994094,20.151264,2*E_n*d**2*k/(h/(2*pi)),13.72*1.377**k*E_n**0.9985*d**1.992/h**0.9997
3,feynman_III_14_14,False,0.981696,46.713531,I_0*(exp(q*Volt/(kb*T))-1),0.1343*3.938**Volt*4.003**q*I_0**0.9987/(T**2....
4,feynman_III_15_12,False,0.248010,4.339468,2*U*(1-cos(k*d)),3.117*U**1.005/(d**0.2333*k**0.1799)


In [14]:
results.to_csv('results/results_full.csv', index=False)

In [15]:
results.sort_values('score', ascending=False).head(50)

,dataset,equal,score,time,original,equation
27,feynman_II_27_16,True,1.000000,4.131348,epsilon*c*Ef**2,1.0*Ef**2.0*c**1.0*epsilon**1.0
28,feynman_II_27_18,True,1.000000,1.051952,epsilon*Ef**2,1.0*Ef**2.0*epsilon**1.0
50,feynman_I_12_1,True,1.000000,1.444628,mu*Nn,1.0*Nn**1.0*mu**1.0
67,feynman_I_25_13,True,1.000000,1.187128,q/C,1.0*q**1.0/C**1.0
57,feynman_I_14_3,True,1.000000,5.615578,m*g*z,1.0*g**1.0*m**1.0*z**1.0
54,feynman_I_12_5,True,1.000000,1.004581,q2*Ef,1.0*Ef**1.0*q2**1.0
88,feynman_I_43_31,True,1.000000,3.929628,mob*kb*T,1.0*T**1.0*kb**1.0*mob**1.0
82,feynman_I_39_1,True,1.000000,1.006085,3/2*pr*V,1.5*V**1.0*pr**1.0
46,feynman_II_8_31,True,1.000000,1.457957,epsilon*Ef**2/2,0.5*Ef**2.0*epsilon**1.0
58,feynman_I_14_4,True,1.000000,1.678720,1/2*k_spring*x**2,0.5*k_spring**1.0*x**2.0


In [16]:
# Filter out datasets containing the word "test"
results = results[~results.dataset.str.contains('test')]

In [17]:
len(results[results['score'] > 0.99])

34

In [18]:
len(results[results['score'] > 0.9])

61

In [19]:
results['score'].mean()

np.float64(0.8186857274992141)

In [21]:
results['score'].std()

np.float64(0.25583422075957657)

In [22]:
results['score'].median()

np.float64(0.9549805103883398)

In [20]:
results['time'].describe()

count      99.000000
mean       42.450882
std       210.733911
min         0.277645
25%         3.766888
50%         5.375650
75%        15.485202
max      2019.514035
Name: time, dtype: float64